In [1]:
#Note - some of these were not built from simple run code so format
from san import *
import san
from decimal import Decimal
import pandas as pd
import datetime
import pickle # pip install pickle

In [10]:
#Getting all slugs/tickers/names/marketsegment/supply and dropping all with missing or 0 supply
data = san.get("projects/all")
data = data.dropna()
data['totalSupply'] = data['totalSupply'].astype(float)
data = data[data['totalSupply'] != 0]

In [11]:
#limiting only to cryptocurrency
crypto = data[data["marketSegment"] == "Cryptocurrency"]
crypto = crypto.sort_values(by = ["totalSupply"], ascending = False)

In [64]:
#import daily return and calculate equal-weighted market returns
dailyret = pd.read_pickle("hourlyreturn.pkl")
#will be used to calculate sum
temphold = dailyret[:]
temphold = temphold.fillna(0)
temphold = temphold.drop(['datetime'], axis = 1)
temphold['sumreturn'] = temphold.sum(axis = 1)
stuff1 = temphold['sumreturn'].tolist()

#will hold count of firms per year that have non-na returns
temp2hold = dailyret[:]
temp2hold = temp2hold.drop(['datetime'], axis = 1)
temp2hold['count'] = temp2hold.count(axis=1)
stuff2 = temp2hold['count'].tolist()

#will be used to calculate daily return
temp3hold = dailyret[:]
temp3hold['sumreturn'] = stuff1
temp3hold['count'] = stuff2
temp3hold['ew_avgret'] = temp3hold['sumreturn']/temp3hold['count']
ew_avgret = temp3hold['ew_avgret'].tolist()

dailyret["ew_avgret"] = ew_avgret

excessret = pd.DataFrame()
excessret['datetime'] = dailyret['datetime']
namelist = []
for name in dailyret.columns.tolist():
    if name != 'datetime':
        namelist.append(name)
for i in namelist:
    slug = i.replace("dret_", '')
    excessname = "ew_excess_" + slug
    excessret[excessname] = dailyret[i]-dailyret['ew_avgret']
excessret.set_index('datetime', inplace=True)

C:\Users\topcue.lee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\topcue.lee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\topcue.lee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [82]:
#add in marketcap and create value weighted returns
marketcap = pd.read_pickle("marketcap_daily.pkl")
marketcap['totcap'] = marketcap.sum(axis = 1)
#Create firm marketcap (as percentage of total)
perccap = pd.DataFrame()
perccap['datetime'] = marketcap['datetime']
namelist = []
for name in marketcap.columns.tolist():
    if name != 'datetime':
        namelist.append(name)
for i in namelist:
    slug = i.replace("_marketcap_usd", '')
    percname = "perc_cap_" + slug
    perccap[percname] = marketcap[i]/marketcap['totcap']
    
perccap.set_index('datetime', inplace=True)

#merge and create value weighted returns

#total_df = pd.merge(excessret, marketcap, how = "left", on = 'datetime')
dailyret.set_index('datetime', inplace = True)

In [42]:
perccap = perccap.drop(columns = ['perc_cap_totcap'])

KeyError: "['perc_cap_totcap'] not found in axis"

In [52]:
a = perccap.unstack().reset_index()
a = a.rename(columns = {"level_0":"Coin", 0: "perc_cap"})
coinlist = []
for i in a['Coin'].tolist():
    coinlist.append(i.replace("perc_cap_", ''))
a["Coin"] = coinlist

In [83]:
b = dailyret.drop(['ew_avgret'], axis = 1)
b = b.unstack().reset_index()
b = b.rename(columns = {"level_0":"Coin", 0: "daily_ret"})
coinlist = []
for i in b['Coin'].tolist():
    coinlist.append(i.replace("dret_", ''))
b["Coin"] = coinlist
b

,Coin,datetime,daily_ret
0,dimecoin,2018-04-01 00:00:00+00:00,NaN
1,dimecoin,2018-04-02 00:00:00+00:00,-0.164717
2,dimecoin,2018-04-03 00:00:00+00:00,-0.032010
3,dimecoin,2018-04-04 00:00:00+00:00,-0.037779
4,dimecoin,2018-04-05 00:00:00+00:00,0.066080
...,...,...,...
159060,bitcoin-bep2,2021-03-28 00:00:00+00:00,0.005961
159061,bitcoin-bep2,2021-03-29 00:00:00+00:00,0.011050
159062,bitcoin-bep2,2021-03-30 00:00:00+00:00,0.049293
159063,bitcoin-bep2,2021-03-31 00:00:00+00:00,-0.024654


In [97]:
#merge daily return and perccap
mergeab = pd.merge(b, a, how = 'left', on = ['Coin', 'datetime'])
mergeab['holdweight'] = mergeab["daily_ret"]*mergeab["perc_cap"]
form2 = mergeab.drop(["daily_ret", "perc_cap"], axis=1)
form2 = form2.groupby(['datetime']).sum()
form2 = form2.rename(columns = {'holdweight':'vw_market'})
mergeab = pd.merge(mergeab, form2, how = 'left', on = 'datetime')
mergeab = mergeab.drop(columns = ['perc_cap', 'holdweight'], axis = 1)
mergeab['vw_excess_ret'] = mergeab['daily_ret'] - mergeab['vw_market']
mergeab = mergeab.drop(columns = ['vw_market'], axis = 1)

In [98]:
c = excessret.drop(['ew_excess_ew_avgret'], axis=1)
c = c.unstack().reset_index()
c = c.rename(columns = {"level_0":"Coin", 0: "ew_excess_ret"})
coinlist = []
for i in c['Coin'].tolist():
    coinlist.append(i.replace("ew_excess_", ''))
c["Coin"] = coinlist


In [102]:
#combine all returns
returns_df = pd.merge(mergeab, c, how = "left", on = ['Coin','datetime'])
returns_df.dropna()

,Coin,datetime,daily_ret,vw_excess_ret,ew_excess_ret
1,dimecoin,2018-04-02 00:00:00+00:00,-0.164717,-0.200264,-0.293305
2,dimecoin,2018-04-03 00:00:00+00:00,-0.032010,-0.098634,-0.119236
3,dimecoin,2018-04-04 00:00:00+00:00,-0.037779,0.047915,0.050101
4,dimecoin,2018-04-05 00:00:00+00:00,0.066080,0.072287,0.039561
5,dimecoin,2018-04-06 00:00:00+00:00,-0.048346,-0.018115,-0.009953
...,...,...,...,...,...
159060,bitcoin-bep2,2021-03-28 00:00:00+00:00,0.005961,0.006224,0.006256
159061,bitcoin-bep2,2021-03-29 00:00:00+00:00,0.011050,-0.021594,-0.127571
159062,bitcoin-bep2,2021-03-30 00:00:00+00:00,0.049293,0.030096,-0.000262
159063,bitcoin-bep2,2021-03-31 00:00:00+00:00,-0.024654,-0.025555,-0.136896


In [103]:
returns_df.to_pickle("returns_df.pkl")